*Copyright (C) 2024 Intel Corporation*<br>

---

# Quadratic Unconstrained Binary Optimization (QUBO) with Lava

This tutorial ramps up users who want to use Lava's solver for quadratic unconstraint binary optimization (QUBO) problems.

## Example application: finding the Maximum Independent Set of a graph

To demonstrate the solver, we use uniformly sampled randomly generated Maximum Independent Set problems on sparse graphs formulated as QUBO problems. The adjacency matrices of the graphs are pre-generated. The following example shows how to formulate and solve a problem as QUBO, using the NP-hard maximum independent set (MIS) problem from graph theory. The goal of an MIS task is to find the largest subset of vertices in a graph that are mutually unconnected. In the following graph, the purple nodes form such a set:

<br>

<img src="https://user-images.githubusercontent.com/86950058/192372990-ec4e5926-463c-4b30-810d-08a896446d8a.png" width="250"/>

The interpretation is:
- There are as many binary variables in the QUBO representation of MIS as the number of vertices in a graph
- If a vertex is included in the maximum independent set, the corresponding binary variable takes the value 1. Otherwise it is 0.
- Two adjacent vertices (connected via an edge) should not be included simultaneously in the MIS. Therefore, such inclusion incurs a penalty via scaled up adjacency matrix, i.e., the Q-matrix.
- Diagonal elements of Q-matrix correspond to -ve values, biasing the QUBO objective function away from 0 (i.e., avoiding an empty MIS).

## Recommended tutorials before starting

- [Installing Lava](https://github.com/lava-nc/lava/blob/main/tutorials/in_depth/tutorial01_installing_lava.ipynb "Tutorial on Installing Lava")

## Set up the environment

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import dataclass
from scipy import sparse

from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi2HwCfg

from lava.magma.compiler.subcompilers.nc.ncproc_compiler import CompilerOptions
from lava.magma.compiler.subcompilers.nc import partitionable

from lava.lib.optimization.solvers.qubo.process import QUBOSolverProcess
from lava.lib.optimization.solvers.qubo.utils.hyperparameters import QUBOHyperparameters
from lava.lib.optimization.solvers.qubo.utils.solver_config import SolverConfig

#### If using SLURM resource manager, set it up

In [2]:
import os
os.environ['SLURM'] = '1'
os.environ['LOIHI_GEN'] = 'N3C1'
os.environ["BOARD"] = 'ncl-og-05'
os.environ["PATH"] += ':/nfs/ncl/bin'

### Helper functions

In [3]:
def random_initial_values(q, seed=42):
    """Generates a random binary vector, to be used as an 
    initial value for a QUBO problem, given a Q-matrix input.

    Parameters
    ----------
    q (numpy.ndarray) : Q-matrix for a QUBO problem

    Returns
    -------
    A binary vector with randomly chosen elements between 
    0 and 1.
    """
    rng = np.random.default_rng(seed)
    return rng.choice([0, 1], q.shape[0]).astype(np.int8)

def create_config(steps, 
                  target_cost, 
                  hyperparams):
    """Creates configuration for QUBO solver.

    Parameters
    ----------
    steps (int) : number of time-steps for which the solver is 
                  executed 
    target_cost (int) : desired target value of the QUBO objective 
    hyperparams (QUBOHyperparameters) : user-defined hyperparameters 
                                        for simulated annealing

    Returns
    -------
    Configuration class for QUBO solver
    """
    config = SolverConfig()
    config.timeout = steps
    config.target_cost = target_cost
    config.readout = 'pio'
    config.hyperparameters = hyperparams
    config.log_level = 20
    return config

def create_solver(q, config):
    """Creates an instance of the QUBO solver

    Parameters
    ----------
    q (numpy.ndarray) : Q-matrix for a QUBO problem
    config (SolverConfig) : Configuration class for a QUBO problem

    Returns
    -------
    An instance of QUBOSolverProcess class configured with 
    user-provided configuration
    """
    solver = QUBOSolverProcess(qubo_matrix=q, solver_config=config)
    solver._log_config.level = config.log_level
    return solver

def run_solver(solver, config):
    """Runs an instance of the QUBO solver.

    Parameters
    ----------
    solver (QUBOSolverProcess) : An instance of the QUBO solver
    config (SolverConfig) : An instance of the configuration for 
                             QUBO solver

    Returns
    -------
    Binary vector that minimizes the QUBO objective function 
    """
    run_cfg = Loihi2HwCfg()
    CompilerOptions.verbose = True  # Print network partitioning information on stdout.
    CompilerOptions.show_resource_count = True  # Use raw number of registers in the partitioning info, instead of %s.
    solver.run(condition=RunSteps(num_steps=config.timeout), run_cfg=run_cfg)
    best_value = solver.best_variable_assignment.get()  # Fetch the neural state from Loihi 2 at the end of a run.
    solver.stop()  # Stop the execution and free-up the hardware resource.
    return best_value

In [4]:
def solve_qubo(
        q_matrix,
        target_cost: int,
        random_seed: int = 2,
        timeout: int = 4000,
        **hyperparams_user,
):
    """Wrapper function to setup and solve a QUBO problem 
    using the QUBO solver using user-definedd simulated 
    annealing hyperparameters and solver configuration.

    Parameters
    ----------
    q_matrix (numpy.ndarray) : Q-matrix for a QUBO problem
    target_cost (int) : desired target value of the QUBO objective 
    random_seed (int, optional) : seed passed to RNG for generating 
                                  random initial solver state
    timeout (int, optional) : number of Loihi 2 time-steps for 
                              which the solver should run
    hyperparams_user (dict, optional) : simulated annealing 
                                        hyperparameters to override 
                                        the defaults.

    Returns
    -------
    best_cost_calculated (int) : lowest QUBO objective function 
                                 value found at the end of the run
    best_solution (numpy.ndarray) : solution to QUBO corresponding 
                                    to `best_cost_calculated`
    """

    hyperparams = QUBOHyperparameters()

    hyperparams.neuron_model = 'sa'
    hyperparams.temperature_max = int(2)
    hyperparams.temperature_min = int(0)
    hyperparams.temperature_delta = int(1)
    hyperparams.exp_temperature = int(1)
    hyperparams.annealing_schedule = 'linear'
    hyperparams.init_value = ('random', random_seed)
    hyperparams.steps_per_temperature = int(timeout//2)

    hyperparams.__dict__.update(hyperparams_user)

    config = create_config(int(timeout), int(target_cost), hyperparams)

    solver = create_solver(q, config)
    best_solution = run_solver(solver, config)
    best_cost_calculated = (
        np.transpose(best_solution)
        @ q_matrix
        @ best_solution
    )
    return best_cost_calculated, best_solution

### Load pre-generated MIS adjacency matrix

All adjacency matrices correspond to sparse random graphs, such that the edges are sampled from a uniform distribution with 5% probability. Therefore, the adjacency matrices have 5% non-zero elements.

A Q-matrix is derived from an adjacency matrix by simply scaling the latter. 

In [ ]:
def load_mis_uniform_q(size=100, seed=0, scale=(8,1)):
    """Loads the adjacency matrix of a graph for which an MIS 
    problem is being solved as QUBO

    The filenames storing the matrices are standardised.

    Parameters
    ----------
    size (int) : size of the Q-matrix, part of filename
    seed (int) : seed used to generate the random graph, part of 
                 filename
    scale (tuple(int, int)) : scaling factors for diagonal and 
                              off-diagonal elements of the Q-matrix
    """
    q = np.loadtxt(f'./data/qubo/workloads/mis-uniform/mis-uniform-{size}-0.05-{seed}.txt', dtype=int)
    q *= scale[0]
    q[np.diag_indices(q.shape[0])] = -scale[1]
    return q

### Set up and Execute the solver

In [5]:
size = 100    # Valid problem sizes for MIS Uniform: 100, 400, 1000
w_scale = 8    # This should be promoted to a model HyperParam, also...
               # technically you could tune w_diag and w_off separately, but
               # in general an 8:1 ratio seems to give best results.

q = load_mis_uniform_q(size=size, scale=(w_scale * 8, w_scale * 1))
q = sparse.csr_matrix(q) if isinstance(q, np.ndarray) else q.copy()
best_cost_calculated, best_solution = solve_qubo(q_matrix=q,
                                                 target_cost=-1000,
                                                 timeout=4000)
print(f'\n********************************************************')
print(f'Found solution with best cost {best_cost_calculated/w_scale}.')
print(f'\n********************************************************')
print(f'The solution is\n{best_solution.reshape(10, 10)}')

Partitioning converged after iteration=1
Per core utilization:
-------------------------------------------------------------------------
| AxonIn |NeuronGr| Neurons|Synapses| AxonMap| AxonMem|  Total |  Cores |
|-----------------------------------------------------------------------|
|     100|       1|       2|     100|       3|       0|     206|       1|
|     200|       1|     100|     370|     400|       0|    1071|       1|
|       0|       1|       1|       0|       1|       0|       3|       1|
|-----------------------------------------------------------------------|
| Total                                                        |       3|
-------------------------------------------------------------------------


/home/sumedhrr/frameworks.ai.nx.nxsdk/nxcore/arch/n3b/n3board.py:77: UserWarning: Loihi generation overriden by environment variable LOIHI_GEN=N3C1
  warnings.warn("Loihi generation overriden by environment variable LOIHI_GEN={}".format(os.environ["LOIHI_GEN"]))


INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 10.54.73.21:36335
INFO:DRV:      Host server up..............Done 0.44s
INFO:DRV:      Mapping chipIds.............Done 0.02ms
INFO:DRV:      Mapping coreIds.............Done 0.10ms
INFO:DRV:      Partitioning neuron groups..Done 1.20ms
INFO:DRV:      Mapping axons...............Done 2.24ms
INFO:DRV:      Configuring Spike Block.....Done 0.01ms
INFO:DRV:      Writes SpikeIO Config to FileDone 0.01ms
INFO:DRV:      Initializes Python MQ.......Done 0.00ms
INFO:DRV:      Partitioning MPDS...........Done 0.90ms
INFO:DRV:      Compiling Embedded snips....Done 0.38s
INFO:DRV:      Compiling Host snips........Done 0.09ms
INFO:DRV:      Compiling Register Probes...Done 0.17ms
INFO:DRV:      Compiling Spike Probes......Done 0.02ms
INFO:HST:  Args chip=0 cpu=0 /home/sumedhrr/frameworks.ai.nx.nxsdk/nxcore/arch/base/pre_execution/../../../../temp/7e0d6a50-50aa-11ef-b608-eb5f7587f812/launcher_chip0_cpu0.bin --chips=1 --remote-rela